In [1]:
import pandas as pd
df1 = pd.read_csv("dfearr0.08_0.06.csv")
df2 = pd.read_csv("df_mfeatures.csv")
df3 = pd.read_csv("BinnedMetaFeatures.csv")

In [5]:
df1 = df1[['File','FeatureAlgo']]

In [8]:
merged_df = df1.merge(df2, on='File').merge(df3, on='File')
merged_df.to_csv("KnowledgeBase.csv", index=False)

In [22]:
df = pd.read_csv("KnowledgeBase.csv")

# Run from here

In [26]:
import pandas as pd
df = pd.read_csv("KnowledgeBase.csv")

In [27]:
df_char_raw = df[[ 'cor.mean', 'cov.mean', 'eigenvalues.mean', 'g_mean.mean', 'h_mean.mean', 'iq_range.mean', 
 'kurtosis.mean', 'mad.mean', 'max.mean', 'mean.mean', 'median.mean', 'min.mean', 'nr_cor_attr', 'nr_norm', 'nr_outliers', 'range.mean', 'sd.mean', 'skewness.mean', 'sparsity.mean',
 't_mean.mean', 'var.mean', 'attr_to_inst', 'inst_to_attr', 'nr_attr', 'nr_bin', 'nr_inst', 'nr_num','attr_conc.mean', 
 'attr_ent.mean', 'nUnique', 'ena', 'snr.mean', 'cEntropy', 'FeatureAlgo']]

df_quality_raw = df[['Completeness', 'Conciseness', 'ClassImbRatio', 'ClassOverlapPerc', 'OutlierPerc', 'LabelIssuesPerc', 'FeatureAlgo']]

df_raw = df[['Completeness', 'Conciseness', 'cor.mean', 'cov.mean', 'eigenvalues.mean', 'g_mean.mean', 'h_mean.mean', 'iq_range.mean', 
 'kurtosis.mean', 'mad.mean', 'max.mean', 'mean.mean', 'median.mean', 'min.mean', 'nr_cor_attr', 'nr_norm', 'nr_outliers', 'range.mean', 'sd.mean', 'skewness.mean', 'sparsity.mean',
 't_mean.mean', 'var.mean', 'ClassImbRatio', 'ClassOverlapPerc', 'OutlierPerc', 'attr_to_inst', 'inst_to_attr', 'nr_attr', 'nr_bin', 'nr_inst', 'nr_num','attr_conc.mean', 
 'attr_ent.mean', 'LabelIssuesPerc','nUnique', 'ena', 'snr.mean', 'cEntropy', 'FeatureAlgo']]


df_char_bin = df[['cor.mean_bins', 'cov.mean_bins',
       'eigenvalues.mean_bins', 'g_mean.mean_bins', 'h_mean.mean_bins',
       'iq_range.mean_bins', 'kurtosis.mean_bins', 'mad.mean_bins',
       'max.mean_bins', 'mean.mean_bins', 'median.mean_bins', 'min.mean_bins',
       'nr_cor_attr_bins', 'nr_norm_bins', 'nr_outliers_bins',
       'range.mean_bins', 'sd.mean_bins', 'skewness.mean_bins',
       'sparsity.mean_bins', 't_mean.mean_bins', 'var.mean_bins',
       'attr_to_inst_bins', 'inst_to_attr_bins',
       'nr_attr_bins', 'nr_inst_bins', 'nr_num_bins', 'nr_bin_bins',
       'attr_conc.mean_bins', 'attr_ent.mean_bins', 'ena_bins',
       'snr.mean_bins', 'cEntropy_bins','FeatureAlgo']]

df_quality_bin = df[['Completeness_bins',
       'Conciseness_bins', 'LabelIssues_bins', 'ClassImbRatio_bins', 'OutlierPerc_bins',
       'ClassOverlapPerc_bins','FeatureAlgo']]

df_bin = df[['Completeness_bins',
       'Conciseness_bins', 'cor.mean_bins', 'cov.mean_bins',
       'eigenvalues.mean_bins', 'g_mean.mean_bins', 'h_mean.mean_bins',
       'iq_range.mean_bins', 'kurtosis.mean_bins', 'mad.mean_bins',
       'max.mean_bins', 'mean.mean_bins', 'median.mean_bins', 'min.mean_bins',
       'nr_cor_attr_bins', 'nr_norm_bins', 'nr_outliers_bins',
       'range.mean_bins', 'sd.mean_bins', 'skewness.mean_bins',
       'sparsity.mean_bins', 't_mean.mean_bins', 'var.mean_bins',
       'LabelIssues_bins', 'ClassImbRatio_bins', 'OutlierPerc_bins',
       'ClassOverlapPerc_bins', 'attr_to_inst_bins', 'inst_to_attr_bins',
       'nr_attr_bins', 'nr_inst_bins', 'nr_num_bins', 'nr_bin_bins',
       'attr_conc.mean_bins', 'attr_ent.mean_bins', 'ena_bins',
       'snr.mean_bins', 'cEntropy_bins', 'FeatureAlgo']]

## RAW DATA

# kNN

In [28]:
import numpy as np 

from sklearn.preprocessing import LabelEncoder

from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix


In [29]:
#k-NN
def evaluate_knn(df, random_state=33):
    df = df.copy()
    df.fillna(0, inplace=True)
    
    X = df.iloc[:, :-1]
    y = df.iloc[:, -1]
    
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)  
    
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    
    X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=random_state)
    
    knn_classifier = KNeighborsClassifier(n_neighbors=5)
    
    cross_val_scores = cross_val_score(knn_classifier, X_train, y_train, cv=kf, scoring='accuracy')
    error_rates = 1 - cross_val_scores
    
    knn_classifier.fit(X_train, y_train)
    y_pred = knn_classifier.predict(X_test)
    
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted', zero_division=0)
    recall = recall_score(y_test, y_pred, average='weighted', zero_division=0)
    f1 = f1_score(y_test, y_pred, average='weighted', zero_division=0)
    conf_matrix = confusion_matrix(y_test, y_pred)
    
    return {
        'Cross-Validation Accuracy': cross_val_scores.mean(),
        'Cross-Validation Error Rate': error_rates.mean(),
        'Test Accuracy': accuracy,
        'Test Error Rate': 1 - accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1 Score': f1,
        'Confusion Matrix': conf_matrix
    }

datasets = {
    'df_raw': df_raw,
    'df_char_raw': df_char_raw,
    'df_quality_raw': df_quality_raw
}

results = {}

for name, df in datasets.items():
    results[name] = evaluate_knn(df)

comparison_df = pd.DataFrame(results).transpose()
print(comparison_df)

               Cross-Validation Accuracy Cross-Validation Error Rate  \
df_raw                          0.514021                    0.485979   
df_char_raw                     0.514021                    0.485979   
df_quality_raw                  0.485185                    0.514815   

               Test Accuracy Test Error Rate Precision    Recall  F1 Score  \
df_raw              0.514286        0.485714  0.491429  0.514286       0.5   
df_char_raw         0.514286        0.485714  0.491429  0.514286       0.5   
df_quality_raw      0.514286        0.485714  0.458242  0.514286  0.484548   

                                                 Confusion Matrix  
df_raw          [[2, 0, 4, 0, 0], [1, 0, 0, 0, 0], [7, 0, 16, ...  
df_char_raw     [[2, 0, 4, 0, 0], [1, 0, 0, 0, 0], [7, 0, 16, ...  
df_quality_raw  [[1, 0, 5, 0, 0], [1, 0, 0, 0, 0], [4, 0, 17, ...  


## Decision trees

In [30]:
#decision

def evaluate_decision_tree(df, random_state=33):
    df = df.copy()
    df.fillna(0, inplace=True)
    
    X = df.iloc[:, :-1]
    y = df.iloc[:, -1]
    
    # Encode labels
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)  
    
    kf = KFold(n_splits=5, shuffle=True, random_state=random_state)
    
    X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=random_state)
    
    tree_classifier = DecisionTreeClassifier(random_state=random_state)
    
    cross_val_scores = cross_val_score(tree_classifier, X_train, y_train, cv=kf, scoring='accuracy')
    error_rates = 1 - cross_val_scores
    
    tree_classifier.fit(X_train, y_train)
    y_pred = tree_classifier.predict(X_test)
    
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted', zero_division=0)
    recall = recall_score(y_test, y_pred, average='weighted', zero_division=0)
    f1 = f1_score(y_test, y_pred, average='weighted', zero_division=0)
    
    conf_matrix = confusion_matrix(y_test, y_pred)
    
    return {
        'Cross-Validation Accuracy': cross_val_scores.mean(),
        'Cross-Validation Error Rate': error_rates.mean(),
        'Test Accuracy': accuracy,
        'Test Error Rate': 1 - accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1 Score': f1,
        'Confusion Matrix': conf_matrix
    }

datasets = {
    'df_raw': df_raw,
    'df_char_raw': df_char_raw,
    'df_quality_raw': df_quality_raw
}

results = {}

for name, df in datasets.items():
    results[name] = evaluate_decision_tree(df)

comparison_df = pd.DataFrame(results).transpose()

print(comparison_df)

               Cross-Validation Accuracy Cross-Validation Error Rate  \
df_raw                          0.521693                    0.478307   
df_char_raw                     0.543386                    0.456614   
df_quality_raw                  0.492063                    0.507937   

               Test Accuracy Test Error Rate Precision    Recall  F1 Score  \
df_raw              0.685714        0.314286  0.691667  0.685714  0.672123   
df_char_raw              0.6             0.4  0.612109       0.6  0.597922   
df_quality_raw      0.742857        0.257143     0.712  0.742857  0.725649   

                                                 Confusion Matrix  
df_raw          [[4, 0, 2, 0, 0], [1, 0, 0, 0, 0], [2, 1, 19, ...  
df_char_raw     [[4, 0, 2, 0, 0], [1, 0, 0, 0, 0], [4, 0, 16, ...  
df_quality_raw  [[3, 1, 2, 0, 0], [0, 0, 0, 0, 1], [1, 1, 21, ...  


## Distance based

In [31]:
results = {}

datasets = {
    'df_raw': df_raw,
    'df_char_raw': df_char_raw,
    'df_quality_raw': df_quality_raw
}

metric_name = 'Euclidean'
metric_func = lambda X_train, X_test: np.sqrt(((X_train.values[:, np.newaxis] - X_test.values) ** 2).sum(axis=2))

kf = KFold(n_splits=5, shuffle=True, random_state=33)

for name, df in datasets.items():
    df.fillna(0, inplace=True)
    
    X = df.iloc[:, :-1]
    y = df.iloc[:, -1]
    
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)
    
    cv_accuracies = []
    test_accuracies = []
    precisions = []
    recalls = []
    f1_scores = []
    confusion_matrices = []
    
    for train_index, test_index in kf.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y_encoded[train_index], y_encoded[test_index]
        
        distances = metric_func(X_train, X_test)
        
        nearest_indices = np.argmin(distances, axis=0)
        
        predicted_labels = y_train[nearest_indices]
        
        cv_accuracy = accuracy_score(y_test, predicted_labels)
        cv_error_rate = 1 - cv_accuracy
        
        cv_accuracies.append(cv_accuracy)
      
    
    X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=33)
    
    distances = metric_func(X_train, X_test)
    nearest_indices = np.argmin(distances, axis=0)
    nearest_labels = y_encoded[nearest_indices]
    predicted_labels = y_train[nearest_indices] 
    precision = precision_score(y_test, predicted_labels, average='weighted', zero_division=0)
    recall = recall_score(y_test, predicted_labels, average='weighted', zero_division=0)
    f1 = f1_score(y_test, predicted_labels, average='weighted', zero_division=0)
    confusion_matrix_result = confusion_matrix(y_test, predicted_labels)
        
    test_accuracy =accuracy_score(y_test, predicted_labels)

    results[name] = {
        'Cross-Validation Accuracy': np.mean(cv_accuracies),
        'Cross-Validation Error Rate': np.mean([1 - acc for acc in cv_accuracies]),
        'Test Accuracy': (test_accuracy),
        'Test Error Rate': (1 - test_accuracy),
        'Precision': precision,
        'Recall': recall,
        'F1 Score':f1,
        'Confusion Matrix': confusion_matrix_result
    }

results_df = pd.DataFrame(results).transpose()

print(results_df)


               Cross-Validation Accuracy Cross-Validation Error Rate  \
df_raw                          0.519664                    0.480336   
df_char_raw                     0.519664                    0.480336   
df_quality_raw                  0.456303                    0.543697   

               Test Accuracy Test Error Rate Precision    Recall  F1 Score  \
df_raw              0.628571        0.371429  0.686364  0.628571  0.636508   
df_char_raw         0.628571        0.371429  0.686364  0.628571  0.636508   
df_quality_raw      0.514286        0.485714  0.555238  0.514286  0.527907   

                                                 Confusion Matrix  
df_raw          [[2, 0, 4, 0, 0], [0, 1, 0, 0, 0], [3, 3, 17, ...  
df_char_raw     [[2, 0, 4, 0, 0], [0, 1, 0, 0, 0], [3, 3, 17, ...  
df_quality_raw  [[3, 0, 3, 0, 0], [0, 1, 0, 0, 0], [3, 2, 14, ...  


## Unification

In [32]:
def unify_features(test_sample, train_data, train_labels):
    test_features = test_sample.values
    matching_indices = np.all(train_data == test_features, axis=1)
    if matching_indices.any():
        matching_labels = train_labels[matching_indices]
        return np.unique(matching_labels)
    return ["unknown"]

def evaluate_unification(df, random_state=33):
    df = df.copy()
    df.fillna(0, inplace=True)
    
    X = df.iloc[:, :-1]
    y = df.iloc[:, -1]
    
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)
    
    kf = KFold(n_splits=5, shuffle=True, random_state=random_state)
    
    cv_accuracies = []
    test_accuracies = []
    precisions = []
    recalls = []
    f1_scores = []
    confusion_matrices = []
    
    for train_index, test_index in kf.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y_encoded[train_index], y_encoded[test_index]
        
        predicted_labels = []
        for i in range(X_test.shape[0]):
            label = unify_features(X_test.iloc[i], X_train, y_train)
            predicted_labels.append(label)
        
        predicted_labels = [item for sublist in predicted_labels for item in sublist]
        
        valid_indices = [i for i, label in enumerate(predicted_labels) if label != "unknown" and i < len(y_test)]
        
        if not valid_indices:
            continue
        
        valid_predictions = np.array([predicted_labels[i] for i in valid_indices])
        valid_truth = np.array([y_test[i] for i in valid_indices])
        
        cv_accuracy = accuracy_score(valid_truth, valid_predictions)
       
        cv_accuracies.append(cv_accuracy)
        
    X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=2)

    predicted_labels = []
    for i in range(X_test.shape[0]):
        label = unify_features(X_test.iloc[i], X_train, y_train)
        predicted_labels.append(label)
    predicted_labels = [item for sublist in predicted_labels for item in sublist]
        
    valid_indices = [i for i, label in enumerate(predicted_labels) if label != "unknown" and i < len(y_test)]
        
    valid_predictions = np.array([predicted_labels[i] for i in valid_indices])
    valid_truth = np.array([y_test[i] for i in valid_indices])
        
    precision = precision_score(valid_truth, valid_predictions, average='weighted', zero_division=0)
    recall = recall_score(valid_truth, valid_predictions, average='weighted', zero_division=0)
    f1 = f1_score(valid_truth, valid_predictions, average='weighted', zero_division=0)
    confusion_matrix_result = confusion_matrix(valid_truth, valid_predictions)    
    test_accuracy = accuracy_score(valid_truth, valid_predictions)

    return {
        'Cross-Validation Accuracy': np.mean(cv_accuracies),
        'Cross-Validation Error Rate': np.mean([1 - acc for acc in cv_accuracies]),
        'Test Accuracy': test_accuracy,
        'Test Error Rate': 1 - test_accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1 Score': f1,
        'Confusion Matrix': confusion_matrix_result
    }

datasets = {
    'df_raw': df_raw,
    'df_char_raw': df_char_raw,
    'df_quality_raw': df_quality_raw
}

results = {}

for name, df in datasets.items():
    results[name] = evaluate_unification(df)

results_df = pd.DataFrame(results).transpose()

print(results_df)

/home/d19125691/.local/lib/python3.6/site-packages/numpy/lib/function_base.py:393: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/home/d19125691/.local/lib/python3.6/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


               Cross-Validation Accuracy Cross-Validation Error Rate  \
df_raw                          0.333333                    0.666667   
df_char_raw                     0.333333                    0.666667   
df_quality_raw                      0.44                        0.56   

               Test Accuracy Test Error Rate Precision Recall  F1 Score  \
df_raw                   NaN             NaN         0      0         0   
df_char_raw              NaN             NaN         0      0         0   
df_quality_raw           0.4             0.6         1    0.4  0.571429   

                                 Confusion Matrix  
df_raw                                         []  
df_char_raw                                    []  
df_quality_raw  [[0, 0, 0], [2, 2, 1], [0, 0, 0]]  


/home/d19125691/.local/lib/python3.6/site-packages/numpy/lib/function_base.py:393: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/home/d19125691/.local/lib/python3.6/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


## BIN data

## kNN

In [33]:
def evaluate_knn(df, random_state=33):
    df = df.copy()
    df.fillna(0, inplace=True)
    
    X = df.iloc[:, :-1]
    y = df.iloc[:, -1]
    
    label_encoder = LabelEncoder()

    for col in X:
        X[col] = label_encoder.fit_transform(X[col])
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)  
    
    kf = KFold(n_splits=5, shuffle=True, random_state=random_state)
    
    X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=random_state)
    
    knn_classifier = KNeighborsClassifier(n_neighbors=5)
    
    cross_val_scores = cross_val_score(knn_classifier, X_train, y_train, cv=kf, scoring='accuracy')
    error_rates = 1 - cross_val_scores
    
    knn_classifier.fit(X_train, y_train)
    y_pred = knn_classifier.predict(X_test)
    
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted', zero_division=0)
    recall = recall_score(y_test, y_pred, average='weighted', zero_division=0)
    f1 = f1_score(y_test, y_pred, average='weighted', zero_division=0)
    conf_matrix = confusion_matrix(y_test, y_pred)
    
    return {
        'Cross-Validation Accuracy': cross_val_scores.mean(),
        'Cross-Validation Error Rate': error_rates.mean(),
        'Test Accuracy': accuracy,
        'Test Error Rate': 1 - accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1 Score': f1,
        'Confusion Matrix': conf_matrix
    }

datasets = {
    'df_bin': df_bin,
    'df_char_bin': df_char_bin,
    'df_quality_bin': df_quality_bin
}

results = {}

for name, df in datasets.items():
    results[name] = evaluate_knn(df)

comparison_df = pd.DataFrame(results).transpose()
print(comparison_df)

               Cross-Validation Accuracy Cross-Validation Error Rate  \
df_bin                          0.484921                    0.515079   
df_char_bin                     0.507407                    0.492593   
df_quality_bin                  0.441534                    0.558466   

               Test Accuracy Test Error Rate Precision    Recall  F1 Score  \
df_bin                   0.6             0.4  0.472381       0.6  0.527126   
df_char_bin         0.542857        0.457143  0.481143  0.542857  0.508333   
df_quality_bin      0.542857        0.457143  0.555195  0.542857  0.539144   

                                                 Confusion Matrix  
df_bin          [[1, 0, 5, 0, 0], [1, 0, 0, 0, 0], [3, 0, 20, ...  
df_char_bin     [[2, 0, 4, 0, 0], [1, 0, 0, 0, 0], [6, 0, 17, ...  
df_quality_bin  [[4, 0, 2, 0, 0], [1, 0, 0, 0, 0], [4, 0, 15, ...  


## Decision tree

In [34]:
def evaluate_decision_tree(df, random_state=33):
    df = df.copy()
    df.fillna(0, inplace=True)
    
    X = df.iloc[:, :-1]
    y = df.iloc[:, -1]
    
    label_encoder = LabelEncoder()

    for col in X:
        X[col] = label_encoder.fit_transform(X[col])
        
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)  
    
    kf = KFold(n_splits=5, shuffle=True, random_state=random_state)
    
    X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=random_state)
    
    tree_classifier = DecisionTreeClassifier(random_state=random_state)
    
    cross_val_scores = cross_val_score(tree_classifier, X_train, y_train, cv=kf, scoring='accuracy')
    error_rates = 1 - cross_val_scores
    
    tree_classifier.fit(X_train, y_train)
    y_pred = tree_classifier.predict(X_test)
    
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted', zero_division=0)
    recall = recall_score(y_test, y_pred, average='weighted', zero_division=0)
    f1 = f1_score(y_test, y_pred, average='weighted', zero_division=0)
    
    conf_matrix = confusion_matrix(y_test, y_pred)
    
    return {
        'Cross-Validation Accuracy': cross_val_scores.mean(),
        'Cross-Validation Error Rate': error_rates.mean(),
        'Test Accuracy': accuracy,
        'Test Error Rate': 1 - accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1 Score': f1,
        'Confusion Matrix': conf_matrix
    }

datasets = {
    'df_bin': df_bin,
    'df_char_bin': df_char_bin,
    'df_quality_bin': df_quality_bin
}

results = {}

for name, df in datasets.items():
    results[name] = evaluate_decision_tree(df)

comparison_df = pd.DataFrame(results).transpose()

print(comparison_df)

               Cross-Validation Accuracy Cross-Validation Error Rate  \
df_bin                          0.456349                    0.543651   
df_char_bin                     0.492063                    0.507937   
df_quality_bin                  0.514286                    0.485714   

               Test Accuracy Test Error Rate Precision    Recall  F1 Score  \
df_bin                   0.4             0.6  0.457895       0.4   0.42449   
df_char_bin         0.514286        0.485714   0.47619  0.514286  0.493131   
df_quality_bin      0.514286        0.485714  0.475429  0.514286  0.494048   

                                                 Confusion Matrix  
df_bin          [[2, 1, 3, 0, 0], [0, 0, 1, 0, 0], [5, 2, 12, ...  
df_char_bin     [[2, 0, 4, 0, 0], [1, 0, 0, 0, 0], [5, 0, 16, ...  
df_quality_bin  [[1, 0, 5, 0, 0], [1, 0, 0, 0, 0], [2, 0, 17, ...  


## Distance based

In [35]:
datasets = {
    'df_bin': df_bin,
    'df_char_bin': df_char_bin,
    'df_quality_bin': df_quality_bin
}
results = {}

metric_name = 'Euclidean'
metric_func = lambda X_train, X_test: np.sqrt(((X_train.values[:, np.newaxis] - X_test.values) ** 2).sum(axis=2))

kf = KFold(n_splits=5, shuffle=True, random_state=33)

for name, df in datasets.items():
    df.fillna(0, inplace=True)
    
    X = df.iloc[:, :-1]
    y = df.iloc[:, -1]
    label_encoder = LabelEncoder()

    for col in X:
        X[col] = label_encoder.fit_transform(X[col])
        
    
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)
    
    cv_accuracies = []
    test_accuracies = []
    precisions = []
    recalls = []
    f1_scores = []
    confusion_matrices = []
    
    for train_index, test_index in kf.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y_encoded[train_index], y_encoded[test_index]
        
        distances = metric_func(X_train, X_test)
        
        nearest_indices = np.argmin(distances, axis=0)
        
        predicted_labels = y_train[nearest_indices]
        
        cv_accuracy = accuracy_score(y_test, predicted_labels)
        cv_error_rate = 1 - cv_accuracy
        
        cv_accuracies.append(cv_accuracy)
      
    
    X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=33)
    
    distances = metric_func(X_train, X_test)
    nearest_indices = np.argmin(distances, axis=0)
    nearest_labels = y_encoded[nearest_indices]
    predicted_labels = y_train[nearest_indices] 
    precision = precision_score(y_test, predicted_labels, average='weighted', zero_division=0)
    recall = recall_score(y_test, predicted_labels, average='weighted', zero_division=0)
    f1 = f1_score(y_test, predicted_labels, average='weighted', zero_division=0)
    confusion_matrix_result = confusion_matrix(y_test, predicted_labels)
        
    test_accuracy =accuracy_score(y_test, predicted_labels)

    results[name] = {
        'Cross-Validation Accuracy': np.mean(cv_accuracies),
        'Cross-Validation Error Rate': np.mean([1 - acc for acc in cv_accuracies]),
        'Test Accuracy': (test_accuracy),
        'Test Error Rate': (1 - test_accuracy),
        'Precision': precision,
        'Recall': recall,
        'F1 Score':f1,
        'Confusion Matrix': confusion_matrix_result
    }

results_df = pd.DataFrame(results).transpose()

print(results_df)

               Cross-Validation Accuracy Cross-Validation Error Rate  \
df_bin                          0.387395                    0.612605   
df_char_bin                     0.468403                    0.531597   
df_quality_bin                  0.380168                    0.619832   

               Test Accuracy Test Error Rate Precision    Recall  F1 Score  \
df_bin              0.485714        0.514286  0.561429  0.485714  0.520809   
df_char_bin         0.428571        0.571429  0.462585  0.428571  0.444555   
df_quality_bin      0.514286        0.485714   0.47619  0.514286  0.490964   

                                                 Confusion Matrix  
df_bin          [[2, 1, 3, 0, 0], [0, 0, 0, 1, 0], [2, 2, 15, ...  
df_char_bin     [[1, 0, 4, 0, 1], [0, 0, 1, 0, 0], [4, 0, 14, ...  
df_quality_bin  [[0, 0, 5, 0, 1], [0, 0, 0, 0, 1], [1, 0, 17, ...  


## Unification

In [36]:
def unify_features(test_sample, train_data, train_labels):
    test_features = test_sample.values
    matching_indices = np.all(train_data == test_features, axis=1)
    if matching_indices.any():
        matching_labels = train_labels[matching_indices]
        return np.unique(matching_labels)
    return ["unknown"]

def evaluate_unification(df, random_state=33):
    df = df.copy()
    df.fillna(0, inplace=True)
    
    X = df.iloc[:, :-1]
    y = df.iloc[:, -1]
    
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)
    
    kf = KFold(n_splits=5, shuffle=True, random_state=random_state)
    
    cv_accuracies = []
    test_accuracies = []
    precisions = []
    recalls = []
    f1_scores = []
    confusion_matrices = []
    
    for train_index, test_index in kf.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y_encoded[train_index], y_encoded[test_index]
        
        predicted_labels = []
        for i in range(X_test.shape[0]):
            label = unify_features(X_test.iloc[i], X_train, y_train)
            predicted_labels.append(label)
        
        predicted_labels = [item for sublist in predicted_labels for item in sublist]
        
        valid_indices = [i for i, label in enumerate(predicted_labels) if label != "unknown" and i < len(y_test)]
        
        if not valid_indices:
            continue
        
        valid_predictions = np.array([predicted_labels[i] for i in valid_indices])
        valid_truth = np.array([y_test[i] for i in valid_indices])
        
        cv_accuracy = accuracy_score(valid_truth, valid_predictions)
       
        cv_accuracies.append(cv_accuracy)
        
    X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=2)
    predicted_labels = []
    for i in range(X_test.shape[0]):
        label = unify_features(X_test.iloc[i], X_train, y_train)
        predicted_labels.append(label)
    predicted_labels = [item for sublist in predicted_labels for item in sublist]
        
    valid_indices = [i for i, label in enumerate(predicted_labels) if label != "unknown" and i < len(y_test)]
        
    valid_predictions = np.array([predicted_labels[i] for i in valid_indices])
    valid_truth = np.array([y_test[i] for i in valid_indices])
        
    precision = precision_score(valid_truth, valid_predictions, average='weighted', zero_division=0)
    recall = recall_score(valid_truth, valid_predictions, average='weighted', zero_division=0)
    f1 = f1_score(valid_truth, valid_predictions, average='weighted', zero_division=0)
    confusion_matrix_result = confusion_matrix(valid_truth, valid_predictions)    
    test_accuracy = accuracy_score(valid_truth, valid_predictions)

    return {
        'Cross-Validation Accuracy': np.mean(cv_accuracies),
        'Cross-Validation Error Rate': np.mean([1 - acc for acc in cv_accuracies]),
        'Test Accuracy': test_accuracy,
        'Test Error Rate': 1 - test_accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1 Score': f1,
        'Confusion Matrix': confusion_matrix_result
    }

datasets = {
    'df_bin': df_bin,
    'df_char_bin': df_char_bin,
    'df_quality_bin': df_quality_bin
}
results = {}

for name, df in datasets.items():
    results[name] = evaluate_unification(df)

results_df = pd.DataFrame(results).transpose()

print(results_df)

               Cross-Validation Accuracy Cross-Validation Error Rate  \
df_bin                          0.395833                    0.604167   
df_char_bin                     0.309051                    0.690949   
df_quality_bin                   0.30827                     0.69173   

               Test Accuracy Test Error Rate Precision    Recall  F1 Score  \
df_bin              0.421053        0.578947  0.336842  0.421053  0.367222   
df_char_bin         0.363636        0.636364  0.483838  0.363636   0.40035   
df_quality_bin      0.205882        0.794118  0.370915  0.205882  0.251859   

                                                 Confusion Matrix  
df_bin          [[1, 0, 1, 0, 0], [0, 0, 1, 0, 0], [4, 0, 6, 1...  
df_char_bin     [[1, 0, 2, 0, 0], [1, 0, 1, 0, 0], [6, 0, 7, 4...  
df_quality_bin  [[1, 0, 1, 0, 2], [0, 0, 1, 1, 0], [4, 2, 5, 5...  


## Standard Deviation

In [37]:
import pandas as pd
df_normalised = pd.read_csv("NormalisedDataset.csv")
df_normalised_quality = df_normalised[['Completeness_normalized', 'Conciseness_normalized',  'ClassImbRatio_normalized', 'ClassOverlapPerc_normalized',
       'OutlierPerc_normalized',  'LabelIssuesPerc_normalized', 'FeatureAlgo_normalized']]
df_normalised_char = df_normalised[[ 'cor.mean_normalized', 'cov.mean_normalized',
       'eigenvalues.mean_normalized', 'g_mean.mean_normalized',
       'h_mean.mean_normalized', 'iq_range.mean_normalized',
       'kurtosis.mean_normalized', 'mad.mean_normalized',
       'max.mean_normalized', 'mean.mean_normalized', 'median.mean_normalized',
       'min.mean_normalized', 'nr_cor_attr_normalized', 'nr_norm_normalized',
       'nr_outliers_normalized', 'range.mean_normalized', 'sd.mean_normalized',
       'skewness.mean_normalized', 'sparsity.mean_normalized',
       't_mean.mean_normalized', 'var.mean_normalized',
        'attr_to_inst_normalized',
       'inst_to_attr_normalized', 'nr_attr_normalized', 'nr_bin_normalized',
       'nr_inst_normalized', 'nr_num_normalized', 'attr_conc.mean_normalized',
       'attr_ent.mean_normalized', 'nUnique_normalized', 'ena_normalized', 'snr.mean_normalized',
       'cEntropy_normalized', 'FeatureAlgo_normalized']]

In [3]:
len(df_normalised.columns)

40

## kNN

In [38]:
def evaluate_knn(df, random_state=33):
    df = df.copy()
    df.fillna(0, inplace=True)
    
    X = df.iloc[:, :-1]
    y = df.iloc[:, -1]
    
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)  
    
    kf = KFold(n_splits=5, shuffle=True, random_state=random_state)
    
    X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=random_state)
    
    knn_classifier = KNeighborsClassifier(n_neighbors=5)
    
    cross_val_scores = cross_val_score(knn_classifier, X_train, y_train, cv=kf, scoring='accuracy')
    error_rates = 1 - cross_val_scores
    
    knn_classifier.fit(X_train, y_train)
    y_pred = knn_classifier.predict(X_test)
    
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted', zero_division=0)
    recall = recall_score(y_test, y_pred, average='weighted', zero_division=0)
    f1 = f1_score(y_test, y_pred, average='weighted', zero_division=0)
    conf_matrix = confusion_matrix(y_test, y_pred)
    
    return {
        'Cross-Validation Accuracy': cross_val_scores.mean(),
        'Cross-Validation Error Rate': error_rates.mean(),
        'Test Accuracy': accuracy,
        'Test Error Rate': 1 - accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1 Score': f1,
        'Confusion Matrix': conf_matrix
    }

datasets = {
    'df_normalised': df_normalised,
    'df_normalised_char': df_normalised_char,
    'df_normalised_quality': df_normalised_quality
}

results = {}

for name, df in datasets.items():
    results[name] = evaluate_knn(df)

comparison_df = pd.DataFrame(results).transpose()
print(comparison_df)

                      Cross-Validation Accuracy Cross-Validation Error Rate  \
df_normalised                          0.506878                    0.493122   
df_normalised_char                      0.53545                     0.46455   
df_normalised_quality                  0.543386                    0.456614   

                      Test Accuracy Test Error Rate Precision    Recall  \
df_normalised              0.571429        0.428571  0.423963  0.571429   
df_normalised_char              0.6             0.4  0.503061       0.6   
df_normalised_quality      0.628571        0.371429  0.438095  0.628571   

                       F1 Score  \
df_normalised          0.486772   
df_normalised_char     0.546779   
df_normalised_quality  0.516327   

                                                        Confusion Matrix  
df_normalised          [[0, 0, 7, 0, 0], [0, 0, 1, 0, 0], [3, 0, 20, ...  
df_normalised_char     [[2, 0, 5, 0, 0], [0, 0, 1, 0, 0], [4, 0, 19, ...  
df_normalised_qu

## Decision tree

In [39]:
def evaluate_decision_tree(df, random_state=33):
    df = df.copy()
    df.fillna(0, inplace=True)
    
    X = df.iloc[:, :-1]
    y = df.iloc[:, -1]

    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)  
    
    kf = KFold(n_splits=5, shuffle=True, random_state=random_state)
    
    X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=random_state)
    
    tree_classifier = DecisionTreeClassifier(random_state=random_state)
    
    cross_val_scores = cross_val_score(tree_classifier, X_train, y_train, cv=kf, scoring='accuracy')
    error_rates = 1 - cross_val_scores
    
    tree_classifier.fit(X_train, y_train)
    y_pred = tree_classifier.predict(X_test)
    
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted', zero_division=0)
    recall = recall_score(y_test, y_pred, average='weighted', zero_division=0)
    f1 = f1_score(y_test, y_pred, average='weighted', zero_division=0)
    
    conf_matrix = confusion_matrix(y_test, y_pred)
    
    return {
        'Cross-Validation Accuracy': cross_val_scores.mean(),
        'Cross-Validation Error Rate': error_rates.mean(),
        'Test Accuracy': accuracy,
        'Test Error Rate': 1 - accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1 Score': f1,
        'Confusion Matrix': conf_matrix
    }

datasets = {
    'df_normalised': df_normalised,
    'df_normalised_char': df_normalised_char,
    'df_normalised_quality': df_normalised_quality
}

results = {}

for name, df in datasets.items():
    results[name] = evaluate_decision_tree(df)

comparison_df = pd.DataFrame(results).transpose()

print(comparison_df)

                      Cross-Validation Accuracy Cross-Validation Error Rate  \
df_normalised                          0.464286                    0.535714   
df_normalised_char                     0.441534                    0.558466   
df_normalised_quality                  0.521164                    0.478836   

                      Test Accuracy Test Error Rate Precision    Recall  \
df_normalised                   0.6             0.4  0.580952       0.6   
df_normalised_char         0.628571        0.371429  0.574429  0.628571   
df_normalised_quality      0.628571        0.371429      0.51  0.628571   

                       F1 Score  \
df_normalised          0.589286   
df_normalised_char     0.600238   
df_normalised_quality  0.557118   

                                                        Confusion Matrix  
df_normalised          [[3, 1, 3, 0, 0], [0, 0, 0, 1, 0], [5, 0, 18, ...  
df_normalised_char     [[3, 1, 3, 0, 0], [0, 0, 1, 0, 0], [3, 0, 19, ...  
df_normalised_qu

## Distance based

In [40]:
datasets = {
    'df_normalised': df_normalised,
    'df_normalised_char': df_normalised_char,
    'df_normalised_quality': df_normalised_quality
}

results = {}

metric_name = 'Euclidean'
metric_func = lambda X_train, X_test: np.sqrt(((X_train.values[:, np.newaxis] - X_test.values) ** 2).sum(axis=2))

kf = KFold(n_splits=5, shuffle=True, random_state=33)

for name, df in datasets.items():
    df.fillna(0, inplace=True)
    
    X = df.iloc[:, :-1]
    y = df.iloc[:, -1]
    
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)
    
    cv_accuracies = []
    test_accuracies = []
    precisions = []
    recalls = []
    f1_scores = []
    confusion_matrices = []
    
    for train_index, test_index in kf.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y_encoded[train_index], y_encoded[test_index]
        
        distances = metric_func(X_train, X_test)
        
        nearest_indices = np.argmin(distances, axis=0)
        
        predicted_labels = y_train[nearest_indices]
        
        cv_accuracy = accuracy_score(y_test, predicted_labels)
        cv_error_rate = 1 - cv_accuracy
        
        cv_accuracies.append(cv_accuracy)
      
    
    X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=33)
    
    distances = metric_func(X_train, X_test)
    nearest_indices = np.argmin(distances, axis=0)
    nearest_labels = y_encoded[nearest_indices]
    predicted_labels = y_train[nearest_indices] 
    precision = precision_score(y_test, predicted_labels, average='weighted', zero_division=0)
    recall = recall_score(y_test, predicted_labels, average='weighted', zero_division=0)
    f1 = f1_score(y_test, predicted_labels, average='weighted', zero_division=0)
    confusion_matrix_result = confusion_matrix(y_test, predicted_labels)
        
    test_accuracy =accuracy_score(y_test, predicted_labels)

    results[name] = {
        'Cross-Validation Accuracy': np.mean(cv_accuracies),
        'Cross-Validation Error Rate': np.mean([1 - acc for acc in cv_accuracies]),
        'Test Accuracy': (test_accuracy),
        'Test Error Rate': (1 - test_accuracy),
        'Precision': precision,
        'Recall': recall,
        'F1 Score':f1,
        'Confusion Matrix': confusion_matrix_result
    }

results_df = pd.DataFrame(results).transpose()

print(results_df)

                      Cross-Validation Accuracy Cross-Validation Error Rate  \
df_normalised                          0.438992                    0.561008   
df_normalised_char                     0.496807                    0.503193   
df_normalised_quality                  0.467395                    0.532605   

                      Test Accuracy Test Error Rate Precision    Recall  \
df_normalised              0.628571        0.371429  0.596296  0.628571   
df_normalised_char         0.628571        0.371429  0.603077  0.628571   
df_normalised_quality      0.428571        0.571429  0.428571  0.428571   

                       F1 Score  \
df_normalised               0.6   
df_normalised_char     0.610573   
df_normalised_quality  0.428571   

                                                        Confusion Matrix  
df_normalised          [[1, 1, 5, 0, 0], [0, 0, 0, 1, 0], [1, 1, 20, ...  
df_normalised_char     [[2, 1, 4, 0, 0], [0, 0, 1, 0, 0], [2, 1, 19, ...  
df_normalised_qu

/home/d19125691/.local/lib/python3.6/site-packages/pandas/core/frame.py:4327: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
/home/d19125691/.local/lib/python3.6/site-packages/pandas/core/frame.py:4327: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


## Unification

In [41]:
def unify_features(test_sample, train_data, train_labels):
    test_features = test_sample.values
    matching_indices = np.all(train_data == test_features, axis=1)
    if matching_indices.any():
        matching_labels = train_labels[matching_indices]
        return np.unique(matching_labels)
    return ["unknown"]

def evaluate_unification(df, random_state=33):
    df = df.copy()
    df.fillna(0, inplace=True)
    
    X = df.iloc[:, :-1]
    y = df.iloc[:, -1]
    
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)
    
    kf = KFold(n_splits=5, shuffle=True, random_state=random_state)
    
    cv_accuracies = []
    test_accuracies = []
    precisions = []
    recalls = []
    f1_scores = []
    confusion_matrices = []
    
    for train_index, test_index in kf.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y_encoded[train_index], y_encoded[test_index]
        
        predicted_labels = []
        for i in range(X_test.shape[0]):
            label = unify_features(X_test.iloc[i], X_train, y_train)
            predicted_labels.append(label)
        
        predicted_labels = [item for sublist in predicted_labels for item in sublist]
        
        valid_indices = [i for i, label in enumerate(predicted_labels) if label != "unknown" and i < len(y_test)]
        
        if not valid_indices:
            continue
        
        valid_predictions = np.array([predicted_labels[i] for i in valid_indices])
        valid_truth = np.array([y_test[i] for i in valid_indices])
        
        cv_accuracy = accuracy_score(valid_truth, valid_predictions)
       
        cv_accuracies.append(cv_accuracy)
        
    X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=2)
    predicted_labels = []
    for i in range(X_test.shape[0]):
        label = unify_features(X_test.iloc[i], X_train, y_train)
        predicted_labels.append(label)
    predicted_labels = [item for sublist in predicted_labels for item in sublist]
        
    valid_indices = [i for i, label in enumerate(predicted_labels) if label != "unknown" and i < len(y_test)]
        
    valid_predictions = np.array([predicted_labels[i] for i in valid_indices])
    valid_truth = np.array([y_test[i] for i in valid_indices])
        
    precision = precision_score(valid_truth, valid_predictions, average='weighted', zero_division=0)
    recall = recall_score(valid_truth, valid_predictions, average='weighted', zero_division=0)
    f1 = f1_score(valid_truth, valid_predictions, average='weighted', zero_division=0)
    confusion_matrix_result = confusion_matrix(valid_truth, valid_predictions)    
    test_accuracy = accuracy_score(valid_truth, valid_predictions)

    return {
        'Cross-Validation Accuracy': np.mean(cv_accuracies),
        'Cross-Validation Error Rate': np.mean([1 - acc for acc in cv_accuracies]),
        'Test Accuracy': test_accuracy,
        'Test Error Rate': 1 - test_accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1 Score': f1,
        'Confusion Matrix': confusion_matrix_result
    }


datasets = {
    'df_normalised': df_normalised,
    'df_normalised_char': df_normalised_char,
    'df_normalised_quality': df_normalised_quality
}

results = {}

for name, df in datasets.items():
    results[name] = evaluate_unification(df)

results_df = pd.DataFrame(results).transpose()

print(results_df)

                      Cross-Validation Accuracy Cross-Validation Error Rate  \
df_normalised                          0.465325                    0.534675   
df_normalised_char                     0.300733                    0.699267   
df_normalised_quality                  0.236842                    0.763158   

                      Test Accuracy Test Error Rate Precision    Recall  \
df_normalised              0.307692        0.692308  0.289377  0.307692   
df_normalised_char         0.333333        0.666667  0.409722  0.333333   
df_normalised_quality      0.242424        0.757576  0.443182  0.242424   

                       F1 Score  \
df_normalised          0.284615   
df_normalised_char     0.358674   
df_normalised_quality  0.306138   

                                                        Confusion Matrix  
df_normalised          [[1, 0, 0, 0, 0], [0, 0, 1, 0, 0], [2, 0, 3, 2...  
df_normalised_char     [[1, 0, 1, 1, 0], [0, 0, 1, 0, 0], [4, 0, 5, 0...  
df_normalised_qu